In [217]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import types
import time
import re
import queue
import string

In [218]:
#Get Seed URLs and URLs from the seed URL
def getQueue(url):
    q = queue.Queue()
    r = requests.get("http://" + url)
    data = r.text
    soup = BeautifulSoup(data)
    for link in soup.find_all('a'):
        link = link.get('href')
        if link!=None:
            q.put(link)
            print (link)
    return q
q=getQueue('en.wikipedia.org/wiki/Machine_learning')

#mw-head
#p-search
/wiki/Machine_Learning_(journal)
/wiki/Data_mining
/wiki/File:Kernel_Machine.svg
/wiki/Statistical_classification
/wiki/Cluster_analysis
/wiki/Regression_analysis
/wiki/Anomaly_detection
/wiki/Association_rule_learning
/wiki/Reinforcement_learning
/wiki/Structured_prediction
/wiki/Feature_engineering
/wiki/Feature_learning
/wiki/Online_machine_learning
/wiki/Semi-supervised_learning
/wiki/Unsupervised_learning
/wiki/Learning_to_rank
/wiki/Grammar_induction
/wiki/Supervised_learning
/wiki/Statistical_classification
/wiki/Regression_analysis
/wiki/Decision_tree_learning
/wiki/Ensemble_learning
/wiki/Bootstrap_aggregating
/wiki/Boosting_(machine_learning)
/wiki/Random_forest
/wiki/K-nearest_neighbors_algorithm
/wiki/Linear_regression
/wiki/Naive_Bayes_classifier
/wiki/Artificial_neural_network
/wiki/Logistic_regression
/wiki/Perceptron
/wiki/Relevance_vector_machine
/wiki/Support_vector_machine
/wiki/Cluster_analysis
/wiki/BIRCH
/wiki/Hierarchical_clustering
/wiki/K-mea

C:\Users\amuth\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\amuth\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [219]:
#Determine if page is relevant (before downloading)
def classRel(url):
    #html = urlopen(url).read()
    #soup = BeautifulSoup(html)
    #text = soup.text()
    return True

In [220]:
#Make sure the url is valid before crawling (get rid of redlink urls)
def makeVal(url):
    if 'redlink' in url:
        return None
    if url[1]=='w':
        url = "http://en.wikipedia.org" + url
        return url
    else:
        return None
#while not q.empty():
    #makeVal(q.get())

In [221]:
#Create a valid file name for the downloaded page
def getName(url):
    name = url.split('/')[-1]
    if '.' in name:
        name = name.split('.')[0]
    return name

In [223]:
#Download pages from the urls in the queue
def getPage(url):
    url = makeVal(url)
    html = urlopen(url).read()
    path='Crawled_Pages/'
    name = getName(url)
    if classRel(url):
        file = open(path+name+'.html','w')
        file.write(str(html))
        file.close()
while not q.empty():
    filenum=0
    url = q.get()
    if makeVal(url) != None & filenum<=100:
        getPage(url)
        filenum = filenum+1
    else:
        continue

IndexError: string index out of range